In [ ]:
import numpy as np
import pandas as pd 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics                           
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from sklearn import datasets, neighbors
from mlxtend.plotting import plot_decision_regions
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, learning_curve)

from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing

# one hot encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import statistics

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import os
import requests
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [ ]:


# Databases used from FAOSTAT about agriculture https://www.fao.org/faostat/en/#data

# Import and export data 
full_data = pd.read_csv("Trade_CropsLivestock.csv",encoding='latin-1')

# Temperature data 
temp = pd.read_csv("temperature_dataset.csv",encoding='latin-1')

In [ ]:
full_data

In [ ]:
temp

## EDA for import and export dataset 

In [ ]:
countries = ["Denmark", "Finland", "Norway", "Ireland"]


filtered_df = full_data[full_data['Area'].isin(countries)]
filtered_df

In [ ]:
# Columns with information about import and export data

full_data = filtered_df[["Area", "Element", "Item", "Year","Unit", "Value"]]
full_data

## EDA for temperature dataset 

In [ ]:
temperature_df = temp[temp['Area'].isin(countries)]
temperature_df

In [ ]:
temp = temperature_df[["Area", "Year", "Element", "Unit", "Value"]]
temp

In [ ]:
temp1 = temp.query("Element=='Temperature change'")

In [ ]:
temp2 = temp.query("Element=='Standard Deviation'")

In [ ]:
temp_merged = pd.merge(temp1, temp2, on=["Area","Year"])

In [ ]:
temp_merged=temp_merged.rename(columns={"Unit_x":"Unit","Value_x":"Temperature","Value_y":"Std_deviation"})

In [ ]:
temp_final = temp_merged[["Area", "Year", "Temperature"]]
temp_final

## Merging import export and temperature datasets

In [ ]:
final_df = pd.merge(full_data, temp_final, on=["Area","Year"])
final_df

## Feature Extraction

In [ ]:
imp_ton = final_df.query("Element=='Import Quantity'")
imp_ton

In [ ]:
imp_value= final_df.query("Element=='Import Value'")
imp_value

In [ ]:
import_df = pd.merge(imp_ton, imp_value, on=["Area","Year","Item","Temperature"])
import_df

In [ ]:
import_list = ["import_quantity", "import_qtd_value", "import_unit_value", "import_value"]
export_list = ["export_quantity", "export_qtd_value", "export_unit_value", "export_value"]

In [ ]:
import_final=import_df.rename(columns={"Unit_x":"import_quantity",
                                       "Value_x":"import_qtd_value",
                                       "Unit_y":"import_unit_value",
                                       "Value_y":"import_value",
                                      })
import_dataset = import_final[["Area", "Year", "Item", "import_quantity", "import_qtd_value", 
                "import_unit_value", "import_value", "Temperature" ]]

import_dataset

In [ ]:
exp_ton = final_df.query("Element=='Export Quantity'")
exp_value = final_df.query("Element=='Export Value'")
export_df = pd.merge(exp_ton, exp_value, on=["Area","Year","Item","Temperature"])
export_df

In [ ]:
export_final=export_df.rename(columns={"Unit_x":"export_quantity",
                                       "Value_x":"export_qtd_value",
                                       "Unit_y":"export_unit_value",
                                       "Value_y":"export_value",
                                      })
export_dataset = export_final[["Area", "Year", "Item", "export_quantity", "export_qtd_value", 
                "export_unit_value", "export_value", "Temperature" ]]

export_dataset

In [ ]:
imp_exp_df = pd.merge(import_dataset, export_dataset, on=["Area","Year","Item","Temperature"])
imp_exp_df

In [ ]:
imp_exp_df = imp_exp_df[["Area", "Year", "Item", "import_quantity", "import_qtd_value", 
                "import_unit_value", "import_value", "export_quantity", "export_qtd_value", 
                "export_unit_value", "export_value", "Temperature" ]]
imp_exp_df

## Handling Missing values

In [ ]:
imp_exp_df.info()

In [ ]:
imp_exp_df.describe()

In [ ]:
imp_exp_df.isnull().sum()

In [ ]:
# Checking the percentage of missing values
round(100*(imp_exp_df.isnull().sum()/len(imp_exp_df.index)), 2)

In [ ]:
# Data missing amount is too small, will remove all 

# Removing NaN rows
imp_exp_df = imp_exp_df[~np.isnan(imp_exp_df['import_qtd_value'])]
imp_exp_df = imp_exp_df[~np.isnan(imp_exp_df['export_qtd_value'])]
imp_exp_df = imp_exp_df[~np.isnan(imp_exp_df['export_value'])]

In [ ]:
round(100*(imp_exp_df.isnull().sum()/len(imp_exp_df.index)), 2)

In [ ]:
# Looking for categorical data
imp_exp_df.nunique(axis = 0)

Now there is no missing values

## Normalising continuous features for our final dataset

In [ ]:
df = imp_exp_df[['import_qtd_value','import_value','export_qtd_value', 'export_value']]

In [ ]:
normalized_df=(df-df.mean())/df.std()

In [ ]:
imp_exp_df = imp_exp_df.drop(['import_qtd_value','import_value','export_qtd_value', 'export_value'], 1)

In [ ]:
imp_exp_df = pd.concat([imp_exp_df,normalized_df],axis=1)

In [ ]:
imp_exp_df

In [ ]:
# Exporting our dataset outside of jupyter notebook

imp_exp_df.to_csv('out.csv')

In [ ]:
# Defining single datasets for each country for further comparison 

ireland_df = imp_exp_df.query("Area=='Ireland'")
denmark_df = imp_exp_df.query("Area=='Denmark'")
norway_df = imp_exp_df.query("Area=='Norway'")
finland_df = imp_exp_df.query("Area=='Finland'")

In [ ]:
data5 = imp_exp_df.copy()
data5['import_value'] = (data5['import_value']-data5['import_value'].mean())/data5['import_value'].std()
data5['import_value'].head()
sns.distplot(data5['import_value']);

In [ ]:
data5 = imp_exp_df.copy()
data5['export_value'] = (data5['export_value']-data5['export_value'].mean())/data5['export_value'].std()
data5['export_value'].head()
sns.distplot(data5['export_value']);

In [ ]:
df_stats = imp_exp_df[["import_value","export_value", "Temperature"]]

In [ ]:
df_stats.info()

## Correlation between features 

In [ ]:
# correlation between features 
correlation = ireland_df.corr()

# plotting heatmap to see the correlation for better understanding
plt.figure(figsize = (9,7))
sns.heatmap(correlation, annot = True, linecolor = "white",lw=0.5, cmap = "Greens")

In [ ]:
#plot the scatter plot of balance and values variable in data
plt.scatter(imp_exp_df.import_value,imp_exp_df.Year)
plt.show()

#plot the scatter plot of balance and values variable in data
imp_exp_df.plot.scatter(x="import_value",y="Year")
plt.show()

In [ ]:
#plot the scatter plot of balance and values variable in data
plt.scatter(imp_exp_df.export_value,imp_exp_df.Year)
plt.show()

#plot the scatter plot of balance and values variable in data
imp_exp_df.plot.scatter(x="export_value",y="Year")
plt.show()

## Classification using Decision Tree

In [ ]:
dataset = imp_exp_df
X = dataset.iloc[:, [9, 11]].values
y = dataset.iloc[:, 0].values

In [ ]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth = 10, random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the test results
y_pred = classifier.predict(X_test)

In [ ]:
# Confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Calculate cm by calling a method named as 'confusion_matrix'
cm = confusion_matrix(y_test, y_pred)

# Call a method heatmap() to plot confusion matrix
sns.heatmap(cm, annot = True)

# print the classification_report based on y_test and y_predict
print(classification_report(y_test, y_pred))

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

## Applying KNN Neighbors Classifier

### With no pca

In [ ]:
X = dataset.iloc[:, [9, 11]].values
y = dataset.iloc[:, 0].values

In [ ]:
scaler = MinMaxScaler()
X=scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=20)

In [ ]:
knn = KNeighborsClassifier(7)
knn.fit(X_train,y_train)
print("Train score before PCA",knn.score(X_train,y_train),"%")
print("Test score before PCA",knn.score(X_test,y_test),"%")

In [ ]:
# Applying PCA function on training and testing set of X component

# Create and initialise an object (pca) by calling a method PCA
pca = PCA(n_components = 2)

# Transform the data into traning and testing
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
 
# Store the explained variance
explained_variance = pca.explained_variance_ratio_

print(explained_variance)

In [ ]:
plt.bar(range(1,len(pca.explained_variance_ ) + 1), pca.explained_variance_ )
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1, len(pca.explained_variance_ ) + 1), pca.explained_variance_,
         c = 'red',
         label = "Cumulative Explained Variance")
plt.legend(loc = 'best')

In [ ]:
pca=PCA(n_components=2)
X_new=pca.fit_transform(X)

In [ ]:
X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, test_size = 0.3, random_state=20, stratify=y)

In [ ]:
knn_pca = KNeighborsClassifier(7)
knn_pca.fit(X_train_new,y_train)
print("Train score after PCA",knn_pca.score(X_train_new,y_train),"%")
print("Test score after PCA",knn_pca.score(X_test_new,y_test),"%")

## Support Vector Machine

In [ ]:
data_df = imp_exp_df[['Area','import_qtd_value','import_value','export_qtd_value', 'export_value']]

In [ ]:
# splitting into X and y
X = data_df.drop("Area", axis = 1)
y = data_df.import_value.astype(int)

In [ ]:
X, y

In [ ]:
from sklearn.preprocessing import scale
# scaling the features
X_scaled = scale(X)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 4)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Model Biulding 

In [ ]:
# using rbf kernel, C=1, default value of gamma

model = SVC(C = 1, kernel='sigmoid')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Model Evaluation Metrics

In [ ]:
# confusion matrix
confusion_matrix(y_true=y_test, y_pred=y_pred)

In [ ]:
# accuracy
print("accuracy", metrics.accuracy_score(y_test, y_pred))

# precision
print("precision", metrics.precision_score(y_test, y_pred, average='micro'))

# recall/sensitivity
print("recall", metrics.recall_score(y_test, y_pred, average='micro'))

## Grid Search CV method to tune the hyperparameters.

In [ ]:
# creating a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 4)

# specify range of hyperparameters
# Set the parameters by cross-validation
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]


# specify model
model = SVC(kernel="rbf")

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = model, 
                        param_grid = hyper_params, 
                        scoring= 'accuracy', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, y_train)                  

In [ ]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
# converting C to numeric type for plotting on x-axis
cv_results['param_C'] = cv_results['param_C'].astype('int')

# # plotting
plt.figure(figsize=(16,6))

# subplot 1/3
plt.subplot(131)
gamma_01 = cv_results[cv_results['param_gamma']==0.01]

plt.plot(gamma_01["param_C"], gamma_01["mean_test_score"])
plt.plot(gamma_01["param_C"], gamma_01["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.01")
plt.ylim([0.80, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

# subplot 2/3
plt.subplot(132)
gamma_001 = cv_results[cv_results['param_gamma']==0.001]

plt.plot(gamma_001["param_C"], gamma_001["mean_test_score"])
plt.plot(gamma_001["param_C"], gamma_001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.001")
plt.ylim([0.80, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')


# subplot 3/3
plt.subplot(133)
gamma_0001 = cv_results[cv_results['param_gamma']==0.0001]

plt.plot(gamma_0001["param_C"], gamma_0001["mean_test_score"])
plt.plot(gamma_0001["param_C"], gamma_0001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.0001")
plt.ylim([0.80, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

In [ ]:
# printing the optimal accuracy score and hyperparameters
best_score = model_cv.best_score_
best_hyperparams = model_cv.best_params_

print("The best test score is {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))

In [ ]:
# specify optimal hyperparameters
best_params = {"C": 100, "gamma": 0.0001, "kernel":"rbf"}

# model
model = SVC(C=100, gamma=0.0001, kernel="rbf")

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# metrics
print(metrics.confusion_matrix(y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(y_test, y_pred))
# print("precision", metrics.precision_score(y_test, y_pred))
# print("sensitivity/recall", metrics.recall_score(y_test, y_pred))

## Regression Models

In [ ]:
# paiwise scatter plot

plt.figure(figsize=(20, 10))
sns.pairplot(cv_results)
plt.show()

In [ ]:
# correlation matrix
cor = cv_results.corr()
cor

## Ridge Regression

In [ ]:
# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}


ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=200]
cv_results.head()

In [ ]:
# plotting mean test and train scores with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
alpha = 15
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
ridge.coef_

In [ ]:
y = dataset['import_value']
X = df.drop(['import_value'],axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

## Linear Regression

In [ ]:
lrm = LinearRegression()
lrm.fit(X_train, Y_train) #fit an OLS model

In [ ]:
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)  #making predictions

In [ ]:
print("R-squared of the model in training set is: {}".format(lrm.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, Y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(Y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_preds_test) / Y_test)) * 100))

## Lasso Regression

In [ ]:
# using GridSearch for parameter optimization
lassoregr = GridSearchCV(Lasso(),
                    param_grid={
                        'alpha': [0.01, 0.1, 1]
                    }, verbose=1)

lassoregr.fit(X_train, Y_train)

lasso = lassoregr.best_estimator_

In [ ]:
# We are making predictions here
y_preds_train = lasso.predict(X_train)
y_preds_test_lasso = lasso.predict(X_test)

print("R-squared of the model in training set is: {}".format(lasso.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso.score(X_test, Y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(Y_test, y_preds_test_lasso)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_preds_test_lasso) / Y_test)) * 100))

## Random Forest


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

In [ ]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, Y_train)

In [ ]:
y_pred_random = regressor.predict(X_test)

In [ ]:

print("R-squared of the model in training set is: {}".format(regressor.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(regressor.score(X_test, Y_test)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_pred_random)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_pred_random) / Y_test)) * 100))

In [ ]:
# pip install prettytable
from prettytable import PrettyTable

In [ ]:
# Create a table to compare de results

from prettytable import PrettyTable
table = [['Statistics', 'SVM', 'Linear Regression', 'Lasso Regression', 'Random Florest','Decision Tree', 'KNN Neighbors'], 
         ['Accuracy', 0.93, 0.55, 0.55, 0.90, 0.42, 0.53], 
         ['R-squared',0, 0.5, 0.50, 0.90, 0, 0],
         ['Root mean', 0, 0.67, 0.67, 0.30, 0, 0],
         ['Mean abs error', 0, 0.51, 0.51, 0.72, 0, 0]]
tab = PrettyTable(table[0])
tab.add_rows(table[1:])
print(tab)